# 1.3 模板Jinja2和Flask-Bootstrap

目前我们还没用html文件,几乎可以说,是个没前端的架子而已,flask的核心是没有模板这些前端的东西的,
但它有个给力好伙伴可以帮他解决前端动态渲染的问题--同一个开发团队制作的模板工具Jinja2,这也是flask安装的
时候会一并安装的.

## Jinjia2

Jinja2和django的模板非常相似,这是[官网](http://jinja.pocoo.org/docs/dev/)这是[文档翻译](http://docs.jinkan.org/docs/jinja2/)

要具体的学习该模板可以自己看文档,我这边只做简单介绍

### 渲染模板

默认情况下,Flask 在程序文件夹中的 templates 子文件夹中寻找模板
要渲染模板需要调用`render_template()`方法

    return render_template('index.html')

> 我们将主页(url根目录)的返回改为一个html文件

In [11]:
%%writefile ../codes/helloworld/templates/index.html
<p>time:{{count}}</p>
<h1>Hello, world! - flask</h1>
<p>this document carries a cookie </p>
<p>answer={{answer}}</p>

Overwriting ../codes/helloworld/templates/index.html


In [12]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g,make_response,abort,redirect,url_for,render_template
from flask.ext.script import Manager
import time

app = Flask(__name__)
app.debug = True
manager = Manager(app)

@app.before_request
def before_request():
        g.time = time.asctime()
        if request.cookies.get('answer'):
            g.answer = request.cookies.get('answer')
        else:
            g.answer = "0"
##新增
@app.errorhandler(404)
def page_not_found(error):
    return "404,page not found!",404,{"a":"af"}
@app.route('/')
def hello():
    response = make_response(render_template('index.html',count=g.time,answer=g.answer))
    response.set_cookie('answer', '42')
    return response

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)

@app.route('/login')
def login():
    abort(404)
@app.route('/infos')
def infos():
    return redirect(url_for('info'))

Overwriting ../codes/helloworld/helloworld.py


In [13]:
!python ../codes/helloworld/manager.py runserver

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 258-121-533
127.0.0.1 - - [19/Jan/2016 14:47:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 14:47:15] "GET /favicon.ico HTTP/1.1" 200 -
^C


### 变量和过滤器:
变量,包括列表,字典,类的实例和过滤器,都用

    {{ }}

来包裹

如:`{{ var }}`,`{{ var|length }}`,`{{var.length}}`

可以通过`app.template_filter()`装饰器定义过滤器,参数为过滤器的名字

我们来自定义一个过滤器,比如说用来把阿拉伯数字转换为中文银行用的繁体数字


### 渲染模板

默认情况下,Flask 在程序文件夹中的 templates 子文件夹中寻找模板
要渲染模板需要调用`render_template()`方法

    return render_template('index.html')

### 变量和过滤器:
变量,包括列表,字典,类的实例和过滤器,都用

    {{ }}

来包裹

如:`{{ var }}`,`{{ var|length }}`,`{{var.length}}`

可以通过`app.template_filter()`装饰器定义过滤器,参数为过滤器的名字

我们来自定义一个过滤器,比如说用来把一个数字自动换算成美元.

In [21]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g,make_response,abort,redirect,url_for,render_template
from flask.ext.script import Manager
import time

app = Flask(__name__)
app.debug = True
manager = Manager(app)

##新增
@app.template_filter('RMBtoUSD')
def RMBtoUSD(text):
    return "$"+str(round(float(text)*0.1520,2))


@app.before_request
def before_request():
        g.time = time.asctime()
        if request.cookies.get('answer'):
            g.answer = request.cookies.get('answer')
        else:
            g.answer = "0"
@app.errorhandler(404)
def page_not_found(error):
    return "404,page not found!",404,{"a":"af"}
@app.route('/')
def hello():
    response = make_response(render_template('index.html',count=g.time,answer=g.answer,money="100"))
    response.set_cookie('answer', '42')
    return response

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)

@app.route('/login')
def login():
    abort(404)
@app.route('/infos')
def infos():
    return redirect(url_for('info'))




Overwriting ../codes/helloworld/helloworld.py


In [22]:
%%writefile ../codes/helloworld/templates/index.html
<p>time:{{count}}</p>
<h1>Hello, world! - flask</h1>
<p>this document carries a cookie </p>
<p>answer={{answer}}</p>
<p>money={{money|RMBtoUSD}}</p>

Overwriting ../codes/helloworld/templates/index.html


In [23]:
!python ../codes/helloworld/manager.py runserver

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 258-121-533
127.0.0.1 - - [19/Jan/2016 15:24:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 15:24:29] "GET /favicon.ico HTTP/1.1" 200 -
^C


### 循环,判断等逻辑语句:

循环,判断等逻辑控制语句都使用

    {% %}

来包裹

例子:输出列表中的全部内容,如果小于5则不输出:


    {% for i in List %}
        {% if i >= 5 %}
            {{ i }}
        {% endif%}
    {% endfor %}



### 模板的继承和调用:

既然是用模板了,那当然要可以继承和调用以实现复用了,

+ 继承用关键字`block`和`extends`

 {% block content %}{% endblock %}

+ 调用关键字`include`


定义一个base.html作为模板的底层调用:

In [36]:
%%writefile ../codes/helloworld/templates/base.html
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN">
<html>
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
        <title>{% block title %}{% endblock %}</title>
    </head>
    <body>
        {% block content %}{% endblock %}
        {% include "blocks/footer.html" %}
    </body>
</html>

Overwriting ../codes/helloworld/templates/base.html


新建文件夹`includes`,添加我们的页脚部:

In [37]:
%%writefile ../codes/helloworld/templates/blocks/footer.html
<footer>
    <p>定义页脚</p>
    <p>作者:黄思喆</p>
</footer>

Overwriting ../codes/helloworld/templates/blocks/footer.html


之后将我们的主页修改成调用模板:

In [38]:
%%writefile ../codes/helloworld/templates/myapp/index.html
{% extends "/base.html" %}
{% block title %}hello world!{% endblock %}
{% block content %}
  <p>time:{{count}}</p>
  <h1>Hello, world! - flask</h1>
  <p>this document carries a cookie </p>
  <p>answer={{answer}}</p>
  <p>money={{money|RMBtoUSD}}</p>
{% endblock %}


Overwriting ../codes/helloworld/templates/myapp/index.html


In [39]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g,make_response,abort,redirect,url_for,render_template
from flask.ext.script import Manager
import time

app = Flask(__name__)
app.debug = True
manager = Manager(app)

##新增
@app.template_filter('RMBtoUSD')
def RMBtoUSD(text):
    return "$"+str(round(float(text)*0.1520,2))


@app.before_request
def before_request():
        g.time = time.asctime()
        if request.cookies.get('answer'):
            g.answer = request.cookies.get('answer')
        else:
            g.answer = "0"
@app.errorhandler(404)
def page_not_found(error):
    return "404,page not found!",404,{"a":"af"}
@app.route('/')
def hello():
    response = make_response(render_template('myapp/index.html',count=g.time,answer=g.answer,money="100"))
    response.set_cookie('answer', '42')
    return response

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)

@app.route('/login')
def login():
    abort(404)
@app.route('/infos')
def infos():
    return redirect(url_for('info'))

Overwriting ../codes/helloworld/helloworld.py


In [40]:
!python ../codes/helloworld/manager.py runserver

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 258-121-533
127.0.0.1 - - [19/Jan/2016 15:44:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 15:44:53] "GET /favicon.ico HTTP/1.1" 200 -
^C


### 宏

宏可以理解为函数

定义宏在`macros/render_comment.html`:

    {% macro render_comment(comment) %}
        <li>{{ comment }}</li>
    {% endmacro %}

调用宏

    {% import 'render_comment.html' as macros %} 
    <ul>
    {% for comment in comments %}
    {{ render_comment.render_comment(comment) }}
    {% endfor %} 
    </ul>




## 引入bootstrap

要想在程序中集成 Bootstrap,显然要对模板做所有必要的改动。不过,更简单的方法是 使用一个名为
`Flask-Bootstrap` 的 Flask 扩展,简化集成的过程。Flask-Bootstrap 使用 pip 安装:

    $ pip install flask-bootstrap

修改myapp.py

In [41]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g,make_response,abort,redirect,url_for,render_template
from flask.ext.script import Manager
from flask.ext.bootstrap import Bootstrap#新增
import time

app = Flask(__name__)
app.debug = True
bootstrap = Bootstrap(app)# 新增
manager = Manager(app)


@app.template_filter('RMBtoUSD')
def RMBtoUSD(text):
    return "$"+str(round(float(text)*0.1520,2))


@app.before_request
def before_request():
        g.time = time.asctime()
        if request.cookies.get('answer'):
            g.answer = request.cookies.get('answer')
        else:
            g.answer = "0"
@app.errorhandler(404)
def page_not_found(error):
    return "404,page not found!",404,{"a":"af"}
@app.route('/')
def hello():
    response = make_response(render_template('myapp/index.html',count=g.time,answer=g.answer,money="100"))
    response.set_cookie('answer', '42')
    return response

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)

@app.route('/login')
def login():
    abort(404)
@app.route('/infos')
def infos():
    return redirect(url_for('info'))

Overwriting ../codes/helloworld/helloworld.py


修改之前的脚部内容:

In [42]:
%%writefile ../codes/helloworld/templates/blocks/footer.html
<footer class="footer">
    <div class="container">
        <p>&copy;Huang Sizhe 2015</p>
        <p>网站地图</p>
        <p>版权声明>BSD le</a>.</p>
    </div>
</footer>

Overwriting ../codes/helloworld/templates/blocks/footer.html


新增导航栏:

In [43]:
%%writefile ../codes/helloworld/templates/blocks/mainnavbar.html
    <nav class="navbar navbar-inverse navbar-fixed-top"  role="navigation">
         <div class="container">
             <div class="navbar-header">
                 <button type="button" class="navbar-toggle" data-toggle="collapse" data-target=".navbar-collapse">
                     <span class="sr-only">切换导航</span>
                     <span class="icon-bar"></span>
                     <span class="icon-bar"></span>
                     <span class="icon-bar"></span>
                 </button>
                 <a class="navbar-brand" href="/">Mysite</a>
             </div>
             <div class="navbar-collapse collapse">
                 <ul class="nav navbar-nav">
                     <li><a href="/">Home</a></li>
                 </ul>
             </div>
         </div>
    </nav>

Writing ../codes/helloworld/templates/blocks/mainnavbar.html


修改base.html

In [44]:
%%writefile ../codes/helloworld/templates/base.html
{% extends "bootstrap/base.html" %}
{% block styles%}
    {{super()}}
    <style>
    body{
        padding-top:50px;
    }

    </style>
{% endblock%}

{% block navbar %}
    {% include 'blocks/mainnavbar.html' %}
{% endblock %}
{% block content %}
    {% include "blocks/footer.html" %}
{% endblock %}



Overwriting ../codes/helloworld/templates/base.html


修改index.html:

In [45]:
%%writefile ../codes/helloworld/templates/myapp/index.html
{% extends "/base.html" %}
{% block title %}hello world!{% endblock %}
{% block content %}

    <div class="container">
        <div class="page-header">
            <p>time:{{ count }}</p>
            <h1>Hello, world! - flask</h1>
        </div>
        <p>answer={{answer}}</p>
        <p>this document carries a cookie </p>
        <p>money={{money|RMBtoUSD}}</p>
        <p>this document carries a cookie </p>
    </div>    
{{ super() }}
{% endblock %}

Overwriting ../codes/helloworld/templates/myapp/index.html


In [46]:
%%writefile ../codes/helloworld/templates/myapp/info.html
{% extends "/base.html" %}
{% block title %}hello world!{% endblock %}
{% block content %}
<div class="container">
    <div class="page-header">
        <p>time:{{ count }}</p>
        <h1>Your browser is :</h1>    
    </div>
    <p>{{agent}}</p>
</div>
    {{ super() }}
{% endblock %}

Writing ../codes/helloworld/templates/myapp/info.html


In [47]:
!python ../codes/helloworld/manager.py runserver

 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 673-134-554
127.0.0.1 - - [19/Jan/2016 16:07:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 16:07:52] "GET /favicon.ico HTTP/1.1" 200 -
^C


是不是好看多了~

"bootstrap/base.html"定义了以下可用于修改的块:


块名|说明
---|---
doc |整个 HTML 文档
html_attribs |`<html>` 标签的属性
html |`<html>` 标签中的内容
head |`<head>` 标签中的内容
title |`<title>` 标签中的内容
metas |一组 `<meta>` 标签
styles |层叠样式表定义
body_attribs |`<body>` 标签的属性
body |`<body>` 标签中的内容
navbar |用户定义的导航条
content |用户定义的页面内容
scripts |文档底部的 JavaScript 声明

## 总结

可以看到flask的模板jinja2和django的还是比较像的

目前用到的包:

包|作用
---|---
flask|flask web框架
flask-script|flask的上下文shell
jinja2|flask的默认模板
flask-bootstrap|flask的bootstrap前端扩展